# Install necessary packages

In [16]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Part 3: Practical Data Preparation

**Objective:** Handle categorical features using One-Hot Encoding and address class imbalance using SMOTE.

## 1. Setup

Import necessary libraries.

In [17]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

## 2. Data Loading

Load the dataset.

In [18]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    dat = pd.read_csv(file_path)
    return dat

## 3. Categorical Feature Encoding

Implement `encode_categorical_features` using `OneHotEncoder`.

In [28]:
def encode_categorical_features(df, column_to_encode='smoker_status'):
    """
    Encode a categorical column using OneHotEncoder.
    
    Args:
        df: Input DataFrame
        column_to_encode: Name of the categorical column to encode
        
    Returns:
        DataFrame with the categorical column replaced by one-hot encoded columns
    """
    # 1. Extract the categorical column
    # 2. Apply OneHotEncoder
    # 3. Create new column names
    # 4. Replace the original categorical column with the encoded columns
    
    cat_col = df[[column_to_encode]]

    encoder = OneHotEncoder(sparse=False, drop='first') 
    encoded_array = encoder.fit_transform(cat_col)

    encoded_col_names = encoder.get_feature_names_out([column_to_encode])

    encoded_df = pd.DataFrame(encoded_array, columns=encoded_col_names, index=df.index)
    df_encoded = df.drop(columns=[column_to_encode]).copy()
    df_encoded = pd.concat([df_encoded, encoded_df], axis=1)

    return df_encoded

## 4. Data Preparation

Implement `prepare_data_part3` to handle the train/test split correctly.

In [29]:
def prepare_data_part3(df, test_size=0.2, random_state=42):
    """
    Prepare data with categorical encoding.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    # YOUR CODE HERE
    # 1. Encode categorical features using the encode_categorical_features function
    # 2. Select relevant features (including the one-hot encoded ones) and the target
    # 3. Split data into training and testing sets
    # 4. Handle missing values
    
    df = encode_categorical_features(df)
    X = df[['age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi', 'smoker_status_no', 'smoker_status_yes']]
    Y = df[['disease_outcome']]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = test_size, random_state = random_state)
    imputer = SimpleImputer(strategy = 'mean')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)
    
    # Placeholder return - replace with your implementation
    return X_train, X_test, y_train, y_test

## 5. Handling Imbalanced Data

Implement `apply_smote` to oversample the minority class.

In [30]:
def apply_smote(X_train, y_train, random_state=42):
    """
    Apply SMOTE to oversample the minority class.
    
    Args:
        X_train: Training features
        y_train: Training target
        random_state: Random seed for reproducibility
        
    Returns:
        Resampled X_train and y_train with balanced classes
    """

    # Apply SMOTE to balance the classes
    
    smote = SMOTE(random_state=random_state)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)
    
    return X_resampled, y_resampled

## 6. Model Training and Evaluation

Train a model on the SMOTE-resampled data and evaluate it.

In [31]:
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    # Initialize and train a LogisticRegression model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    return model  # Replace with actual implementation

def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """
    # 1. Generate predictions
    # 2. Calculate metrics: accuracy, precision, recall, f1, auc
    # 3. Create confusion matrix
    # 4. Return metrics in a dictionary
    
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] 

    # 2. Calculate metrics
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_prob),
        "confusion_matrix": confusion_matrix(y_test, y_pred).tolist()
    }

    return metrics
    
    

## 7. Save Results

Save the evaluation metrics to a text file.

In [32]:
def save_results(metrics: dict):
    os.makedirs('results', exist_ok=True)
    formatted_metrics = [f"{key}: {value}" for key, value in metrics.items()]

    output_file = 'results/results_part3.txt'
    with open(output_file, 'w') as f:
        for line in formatted_metrics:
            f.write(line + '\n')
    print(f"\nMetrics saved to {output_file}")

## 9. Compare Results

Implement a function to compare model performance between balanced and imbalanced data.

In [ ]:
def compare_models(part1_metrics, part3_metrics):
    """
    Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
    Args:
        part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
        part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
    Returns:
        Dictionary with metric names as keys and improvement percentages as values
    """
    # 1. Calculate percentage improvement for each metric
    # 2. Handle metrics where higher is better (most metrics) and where lower is better
    # 3. Return a dictionary with metric names and improvement percentages
    
    improvement = {
        'accuracy' : (part3_metrics['accuracy'] - part1_metrics['accuracy']) / part1_metrics['accuracy'] * 100,
        'precision' : (part3_metrics['precision'] - part1_metrics['precision']) / part1_metrics['precision'] * 100,
        'recall' : (part3_metrics['recall'] - part1_metrics['recall']) / part1_metrics['recall'] * 100,
        'f1' : (part3_metrics['f1'] - part1_metrics['f1']) / part1_metrics['f1'] * 100,
        'auc' : (part3_metrics['auc'] - part1_metrics['auc']) / part1_metrics['auc'] * 100
    }
    return improvement

## 8. Main Execution

Run the complete workflow.

In [35]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data with categorical encoding
    X_train, X_test, y_train, y_test = prepare_data_part3(df)
    
    # 3. Apply SMOTE to balance the training data
    X_train_resampled, y_train_resampled = apply_smote(X_train, y_train)
    
    # 4. Train model on resampled data
    model = train_logistic_regression(X_train_resampled, y_train_resampled)
    
    # 5. Evaluate on original test set
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 6. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 7. Save results
    save_results(metrics)
    
    # 8. Load Part 1 results for comparison
    import json
    try:
        with open('results/results_part1.txt', 'r') as f:
            part1_metrics = json.load(f)
        
        # 9. Compare models
        comparison = compare_models(part1_metrics, metrics)
        print("\nModel Comparison (improvement percentages):")
        for metric, improvement in comparison.items():
            print(f"{metric}: {improvement:.2f}%")
    except FileNotFoundError:
        print("Part 1 results not found. Run part1_introduction.ipynb first.")

accuracy: 0.8274
precision: 0.3401
recall: 0.8182
f1: 0.4805
auc: 0.9089

Metrics saved to results/results_part3.txt


/Users/jessica/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jessica/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# def compare_models(part1_metrics, part3_metrics):
#     """
#     Calculate percentage improvement between models trained on imbalanced vs. balanced data.
    
#     Args:
#         part1_metrics: Dictionary containing evaluation metrics from Part 1 (imbalanced)
#         part3_metrics: Dictionary containing evaluation metrics from Part 3 (balanced)
        
#     Returns:
#         Dictionary with metric names as keys and improvement percentages as values
#     """
#     # 1. Calculate percentage improvement for each metric
#     # 2. Handle metrics where higher is better (most metrics) and where lower is better
#     # 3. Return a dictionary with metric names and improvement percentages
    
#     improvement = {
#         'accuracy' : (part3_metrics['accuracy'] - part1_metrics['accuracy']) / part1_metrics['accuracy'] * 100,
#         'precision' : (part3_metrics['precision'] - part1_metrics['precision']) / part1_metrics['precision'] * 100,
#         'recall' : (part3_metrics['recall'] - part1_metrics['recall']) / part1_metrics['recall'] * 100,
#         'f1' : (part3_metrics['f1'] - part1_metrics['f1']) / part1_metrics['f1'] * 100,
#         'auc' : (part3_metrics['auc'] - part1_metrics['auc']) / part1_metrics['auc'] * 100
#     }
    
#     # Placeholder return - replace with your implementation
#     return improvement